In [2]:
import pandas as pd, ipywidgets as widgets, numpy as np, matplotlib.pyplot as plt, io
from adjustText import adjust_text
raw_uploader, eln_uploader = widgets.FileUpload(multiple = False), widgets.FileUpload(multiple = False)
display("Upload iQue3 Data (csv)")
display(raw_uploader)
display("Upload ELN w/ Antibodies Sheet (xlsx)")
display(eln_uploader)

'Upload iQue3 Data (csv)'

FileUpload(value={}, description='Upload')

'Upload ELN w/ Antibodies Sheet (xlsx)'

FileUpload(value={}, description='Upload')

In [3]:
iquename, elnname = list(raw_uploader.value.keys())[0], list(eln_uploader.value.keys())[0]
iquedata = pd.read_csv(io.BytesIO(raw_uploader.value[iquename]['content']))
eln = pd.read_excel(io.BytesIO(eln_uploader.value[elnname]['content']))
ab_sheet = pd.read_excel(io.BytesIO(eln_uploader.value[elnname]['content']), sheet_name = "Antibody Plate Map")
iquedata

,Experiment,Analysis,Plate,Export Date and Time,Well ID,Row,Column,Well Type,Count of All Wells,Median BL1-H of singlets,Median RL1-H of singlets,Count of singlets
0,22_05_24 LGR4-5 Primary Flow,Analysis 1,Plate 1,5/24/22 15:13,A01,A,1,LGR4,3615,72579.0,309284.0,1785
1,22_05_24 LGR4-5 Primary Flow,Analysis 1,Plate 1,5/24/22 15:13,B01,B,1,LGR4,3654,79742.0,259030.0,1699
2,22_05_24 LGR4-5 Primary Flow,Analysis 1,Plate 1,5/24/22 15:13,C01,C,1,LGR4,3023,75343.0,981041.0,941
3,22_05_24 LGR4-5 Primary Flow,Analysis 1,Plate 1,5/24/22 15:13,D01,D,1,LGR4,2012,55023.5,1230853.5,484
4,22_05_24 LGR4-5 Primary Flow,Analysis 1,Plate 1,5/24/22 15:13,E01,E,1,LGR4,3927,62467.5,277372.5,948
...,...,...,...,...,...,...,...,...,...,...,...,...
283,22_05_24 LGR4-5 Primary Flow,Analysis 1,Plate 1,5/24/22 15:13,L22,L,22,Negative,1207,11271.0,16096.0,297
284,22_05_24 LGR4-5 Primary Flow,Analysis 1,Plate 1,5/24/22 15:13,M22,M,22,Negative,898,11067.0,16939.0,429
285,22_05_24 LGR4-5 Primary Flow,Analysis 1,Plate 1,5/24/22 15:13,N22,N,22,Negative,554,10154.5,15461.5,52
286,22_05_24 LGR4-5 Primary Flow,Analysis 1,Plate 1,5/24/22 15:13,O22,O,22,Negative,2448,11486.0,17924.0,1362


In [4]:
data_cleaned = iquedata[["Plate", "Well ID", "Well Type", "Median RL1-H of singlets"]]
data_cleaned

,Plate,Well ID,Well Type,Median RL1-H of singlets
0,Plate 1,A01,LGR4,309284.0
1,Plate 1,B01,LGR4,259030.0
2,Plate 1,C01,LGR4,981041.0
3,Plate 1,D01,LGR4,1230853.5
4,Plate 1,E01,LGR4,277372.5
...,...,...,...,...
283,Plate 1,L22,Negative,16096.0
284,Plate 1,M22,Negative,16939.0
285,Plate 1,N22,Negative,15461.5
286,Plate 1,O22,Negative,17924.0


In [8]:
#Return data frame with only samples of the well type specified
def clean_for_well_type(well_type_string):
    return data_cleaned[data_cleaned["Well Type"] == well_type_string]

#Average function that takes a string indicating the well type of interest (marked in iQue3 software, or manually changed
#Separates n=1 and n=2 replicates from 384 well format, resets indeces, and averages
#Returns a series with the averaged MFI ratios, removing the replicate wells
def average_MFI(well_type_string):
    selected_wells = clean_for_well_type(well_type_string)
    MFIs = selected_wells["Median RL1-H of singlets"]
    firsts = MFIs.iloc[::2]
    seconds = MFIs.iloc[1::2]
    firsts.reset_index(inplace = True, drop = True)
    seconds.reset_index(inplace = True, drop = True)
    average = firsts + seconds
    average = average/2
    return average

#Calculates the MFI ratio given a positive and negative series, 
#and appends the MFI ratio to the original data frame, excluding the negative population
#positive_series: the series of the positive values
#negative_series: the series of the negative (untransfected) values
#positive_string: the string that marks the positive samples in the Well Type column from iQue3 csv
def calculate_MFI_Ratio(positive_series, negative_series, positive_string):
    MFI_ratio = positive_series/negative_series
    original_positive = clean_for_well_type(positive_string)
    original_positive = original_positive[::2]
    original_positive.reset_index(inplace = True, drop = True)
    original_positive["MFI Ratio"] = MFI_ratio
    MFIs_Antibody = pd.concat([original_positive, ab_sheet], axis = 1)
    MFIs_Antibody.index += 1
    return MFIs_Antibody[["Plate", "Well ID", "Well Type", "MFI Ratio", "DNA", "Protein Concentration"]] 

In [11]:
pos_string = "LGR5"
averagePositive = average_MFI(pos_string)
averageNegative = average_MFI("Negative")

In [12]:
#Create new DF for MFI ratios(don't modify originals)
MFI_Ratios_Antibody = calculate_MFI_Ratio(averagePositive, averageNegative, pos_string)
MFI_Ratios_Antibody
MFI_Ratios_Antibody.to_excel(pos_string + "Well_ID_MFI_Ratio.xlsx", sheet_name='Analyzed Data')
AllLeadsSorted = MFI_Ratios_Antibody.sort_values(by="MFI Ratio", ascending = False)
AllLeadsSorted

,Plate,Well ID,Well ID,Well Type,MFI Ratio,DNA,Protein Concentration
2,Plate 1,C09,B1,LGR5,7.328859,LRCL1Fc_IgG1,0.17
48,Plate 1,O14,H6,LGR5,6.091651,Tmab_220314_AM,4.27
14,Plate 1,K10,F2,LGR5,3.022793,DKK2Fc_IgG1,0.13
47,Plate 1,M14,G6,LGR5,2.291029,Tmab_220314_AM,4.41
8,Plate 1,O09,H1,LGR5,1.970422,DKK1Fc_IgG1,0.65
1,Plate 1,A09,A1,LGR5,1.931354,CLPSFc_IgG1,3.71
24,Plate 1,O11,H3,LGR5,1.844621,EDIL3_CLPSFc_IgG8,1.36
6,Plate 1,K09,F1,LGR5,1.668884,DKK3Fc_IgG1,0.68
46,Plate 1,K14,F6,LGR5,1.555233,Tmab_220314_AM,4.30
11,Plate 1,E10,C2,LGR5,1.457318,KRA99Fc_IgG1,0.18


In [ ]:
# #Sort DF by highest MFI ratio (top hits)
# AllLeadsSorted = MFIRatio_Antibody_cleaned.sort_values(by="MFI Ratio", ascending = False)
# Top10 = MFIRatio_Antibody_cleaned.nlargest(10, "MFI Ratio")
# #Top10 = Top10.append(MFIRatio_Antibody_cleaned.loc[MFIRatio_Antibody_cleaned["DNA"] == "SC-175-017"])
# fig, ax = plt.subplots()
# ax.scatter(x = AllLeadsSorted["Protein Concentration (ug/uL)"], y=AllLeadsSorted["MFI Ratio"], alpha = .2, color = "blue")
# ax.scatter(x = Top10["Protein Concentration (ug/uL)"], y = Top10["MFI Ratio"], alpha = .8, color = "blue", label = "Top Hits")
# ax.set_title("CD40 Primary Screen")
# ax.set_xlabel("Concentration (mg/ml)")
# ax.set_ylabel("MFI Ratio \n Transfected/Untransfected")
# ax.legend()

# texts = []
# for row in Top10.itertuples(index = False):
#     ab, conc, MFI = row[4], row[5], row[3]
#     ax.annotate(row[4], xy = (conc, MFI), textcoords = "offset points", xytext = (8, -2), ha ='left')
#     #texts.append(plt.text(ab, conc, MFI))